In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# Visualizing The Structure of Your Model

OpenMDAO models can have deep hierarchies of groups and components, with many connections between them.
Often times, it is helpful to visualize the model structure. OpenMDAO provides a model visualization tool
that is accessible via the [openmdao](om-command-n2) command line tool.

For example, if you had built the Sellar problem with a missing connection, and you figured out that there
was a missing connection by running [check setup](../command_line/check_setup),
then you might want to take a closer look at the model to help figure out what's going on.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src3", get_code("openmdao.test_suite.components.sellar.SellarDis1"), display=False)

:::{Admonition} `SellarDis1` class definition 
:class: dropdown

{glue:}`code_src3`
:::

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src4", get_code("openmdao.test_suite.components.sellar.SellarDis2"), display=False)

:::{Admonition} `SellarDis2` class definition 
:class: dropdown

{glue:}`code_src4`
:::

In [ ]:
import numpy as np
import openmdao.api as om

from openmdao.test_suite.components.sellar import SellarDis1, SellarDis2


class SellarMDAConnect(om.Group):

    def setup(self):
        cycle = self.add_subsystem('cycle', om.Group(), promotes_inputs=['x', 'z'])
        cycle.add_subsystem('d1', SellarDis1(), promotes_inputs=['x', 'z'])
        cycle.add_subsystem('d2', SellarDis2(), promotes_inputs=['z'])
        cycle.connect('d1.y1', 'd2.y1')

        ######################################
        # This is a "forgotten" connection!!
        ######################################
        #cycle.connect('d2.y2', 'd1.y2')

        cycle.set_input_defaults('x', 1.0)
        cycle.set_input_defaults('z', np.array([5.0, 2.0]))

        # Nonlinear Block Gauss Seidel is a gradient free solver
        cycle.nonlinear_solver = om.NonlinearBlockGS()

        self.add_subsystem('obj_cmp', om.ExecComp('obj = x**2 + z[1] + y1 + exp(-y2)',
                                                  z=np.array([0.0, 0.0]), x=0.0),
                           promotes_inputs=['x', 'z'])

        self.add_subsystem('con_cmp1', om.ExecComp('con1 = 3.16 - y1'))
        self.add_subsystem('con_cmp2', om.ExecComp('con2 = y2 - 24.0'))

        self.connect('cycle.d1.y1', ['obj_cmp.y1', 'con_cmp1.y1'])
        self.connect('cycle.d2.y2', ['obj_cmp.y2', 'con_cmp2.y2'])


prob = om.Problem()

prob.model = SellarMDAConnect()

prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
prob.driver.options['tol'] = 1e-8

prob.set_solver_print(level=0)

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z', lower=0, upper=10)
prob.model.add_objective('obj_cmp.obj')
prob.model.add_constraint('con_cmp1.con1', upper=0)
prob.model.add_constraint('con_cmp2.con2', upper=0)

prob.setup()

prob.set_val('x', 2.0)
prob.set_val('z', [-1., -1.])

prob.run_driver()
print('minimum found at')
print(prob.get_val('x')[0])
print(prob.get_val('z'))
print('minumum objective')
print(prob.get_val('obj_cmp.obj')[0])

You can generate a visualization of the model in $N^2$ form with the following command:
```
openmdao n2 sellar.py
```


In [ ]:
om.n2(prob, show_browser=False, embeddable=True)

This diagram is a version of a design-structure matrix, with the model hierarchy displayed on the left side.
On the diagonal is each input and output of each of the components. Off-diagonal blocks indicate data connections.
Feed-forward connections are shown in the upper triangle, and feed-back connections (the kind that cause cycles) are shown in the lower triangle.
If you hover over any of the blocks on the diagonal, the incoming and outgoing connections are highlighted with arrows.

The unconnected input mentioned above, `cycle.d1.y2`, is highlighted in red to immediately draw attention to a possible problem.
You can also see that it's a member of the cycle group (in this case, that was obvious by its name, but with variable promotion that will not always be the case).
Furthermore, you can see that the variable you would want to connect `cycle.d1.y1` to, `cycle.d2.y2`, is in the same group.
The lack of cycles in the model is made visually evident by the lack of any connections in the lower triangle of the diagram.

Collapse groups down by right-clicking on them, and zoom in and out of different parts of the hierarchy by left-clicking.
These controls help show that the $N^2$ diagram is a powerful tool for letting you inspect and understand large, complex models.

The right side of the diagram shows the solver hierarchy. Initially, it shows the linear solvers but by clicking on
the `Toggle Solver Names` button in the toolbar, the names of the nonlinear solvers can be displayed.